## The dataset is a webscraped from IMDB website using BeautifulSoup.

In [1]:
from requests import get

In [2]:
url = "http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1"

###### Say if you want to scrape a list of 1000 movies, we would have to send in 1000 requests to the website. Assuming each request takes 1 second to execute, it would take a 1000 seconds to execute. When we explore the website a bit we find noval ways which will help us to execute our scraping much more faster.

#### Inspecting the link
Exploring the IMDB site for a while we find that while using advanced search feature we can have look at the best movies in the given time frame by 50 movies per page. This reduces our time 50 times as we can extract 50 movies per request.

further let's explore the url to have a better understanding of waht is happeninng on each request. The link has following elements:

release_date: this is takes in the value for the year we are interested in. (2017 in our case)
sort: this takes in the value by which we want to sort our list. (num_votes,desc in our case, desc suggests descending order)
page: this takes the page number we are interested in. (1 in our case)
Further when you click on the next tab, we get an additional element in the link: "http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=2&ref_=adv_nxt"

ref: this suggests if we want to go on the next or the previous page.

In [3]:
response = get(url)
response

<Response [200]>

###### Response 200 means that we are successfull in obtaining data from the website.. if it is other than 200 than you are not able to fetch the data from the website

Understanding the HTML structure of the page
As we can see get() method pulls up the html document for the url. To further pull out the data we want we need to closely inspect the HTML content of the page. We can turn to the developer tools of chrome for this, cntrl+shift+i or simply right clicking on the element you want to study and seelcting the inspect element will pull up that data set for you. (This is performed on chrome bust should work with other browsers as well.)

We observe that there is a div tag for each movie. Thus, we can simply loop through the 50 dev tags assosicated with each movie using BeautifulSoup.

In [4]:
from bs4 import BeautifulSoup as bs ## importing Beautiful Soup
html_soup = bs(response.text,'html.parser')## python's in built library HTML parser
type(html_soup)

bs4.BeautifulSoup

###### Before extracting 50 div containers, we need to figure out what makes them unique. While exploring the HTML content we find that the class attribute has two values "lister-item" and "mode-advanced". This combination is unique to all the div containers.

In [5]:
id_check = html_soup.find(id ="main")
movie_container = id_check.find_all(class_ ="lister-item mode-advanced")
len(movie_container)

50

In [6]:
container = movie_container[0]
meta = container.find("span", class_ = "metascore").text
print(meta)

77        


###### find
find_all() returned a ResultSet object with the length of 50 movies we are interested in.

Now, we'll select the conatainers one by one and extract the elements of our interest like:

The name of movie.
The year of release.
The IMDB rating.
The metascore.
The number of votes etc...

In [7]:
#list to store scraped value data in:
movie_names = []
year_release = []
imdb_ratings = []
metascores = []
votes = []
movie_description = []
certificate = []
runtime = []
genre = []
director_name = []
star_cast = []
gross_value = []

#extract data from individual movie container
for container in movie_container:
    
   
    #if movie has Metascore, then extract:
    if container.find("div", class_ = "ratings-metascore") is not None:
        
        
        #the movie_name
        name = container.h3.a.text
        movie_names.append(name)
        
        #the year od release
        release = container.find("span", class_ = "lister-item-year text-muted unbold").text
        year_release.append(release)
        
        #the ratings for the movies
        ratings = float(container.strong.text)
        imdb_ratings.append(ratings)
        
        #the metascores
        meta = container.find("span", class_ = "metascore").text
        metascores.append(int(meta))
        
        #the votes
        vote = container.find("span", attrs = {"name":"nv"})['data-value']
        votes.append(int(vote))
        
        #the certificate
        certi = container.find("span", class_ = "certificate").text
        certificate.append(certi)
        
        #the runtime
        run = container.find("span", class_ ="runtime").text
        runtime.append(run)
        
        #the genre
        gen = container.find("span", class_ ="genre").text
        genre.append(gen)
        
        #fetching all <p> tags
        content = container.find_all("p")
        
        #the description
        desc = content[1].text
        movie_description.append(desc)
        
        #subsetting all the <a> tags in 3rd <p> tag
        content_2 = content[2].find_all("a")
        
        #the director
        director = content_2[0].text
        director_name.append(director)
        
        #the gross value
        if len(container.find_all("span", attrs = {"name":"nv"})) >= 2:
            gross = container.find_all("span", attrs = {"name":"nv"})[1]['data-value']
            gross_value.append(gross)
        else:
            gross_value.append(None)
        #extracting artists names
        temp = []
        for i in range(len(content_2)-1):
            temp.append(content_2[i].text)
        star_cast.append(temp)

In [8]:

import pandas as pd

test_df = pd.DataFrame({"movie_names":movie_names,
                        "year_release":year_release,
                        "imdb_ratings":imdb_ratings,
                        "metscores":metascores,
                        "votes":votes,
                        "movie_description":movie_description,
                        "certificate":certificate,
                        "runtime":runtime,
                        "genre":genre,
                        "director_name": director_name,
                        "star_cast": star_cast,
                        "gross_value":gross_value
                       })

In [9]:
test_df

certificate        director_name  \
0            R        James Mangold   
1        PG-13        Patty Jenkins   
2        PG-13    Christopher Nolan   
3        PG-13           James Gunn   
4        PG-13         Rian Johnson   
5        PG-13            Jon Watts   
6            R         Jordan Peele   
7            R         Edgar Wright   
8        PG-13        Taika Waititi   
9            R     Denis Villeneuve   
10           R      Andy Muschietti   
11           R       Chad Stahelski   
12       PG-13          Zack Snyder   
13          PG          Bill Condon   
14       PG-13  Jordan Vogt-Roberts   
15           R         Ridley Scott   
16       PG-13      Joachim Rønning   
17           R      Martin McDonagh   
18           R       Matthew Vaughn   
19       PG-13          Matt Reeves   
20           R   Guillermo del Toro   
21           R      Daniel Espinosa   
22       PG-13       Rupert Sanders   
23       PG-13          Guy Ritchie   
24          PG          Lee Unkrich   
25       PG-13        Alex Kurtzman   
26           R       Patrick Hughes   
27       TV-MA           David Ayer   
28       PG-13      Kenneth Branagh   
29           R         David Leitch   
30           R      Taylor Sheridan   
31           R     Darren Aronofsky   
32       PG-13           Luc Besson   
33       PG-13          Jake Kasdan   
34          PG          Chris McKay   
35           R          Seth Gordon   
36           R         Greta Gerwig   
37       PG-13          Michael Bay   
38           R           Doug Liman   
39       PG-13        Nikolaj Arcel   
40           R      Luca Guadagnino   
41          PG       Michael Gracey   
42           R          Danny Boyle   
43       PG-13       Dean Israelite   

                                         genre  gross_value  imdb_ratings  \
0          \nAction, Drama, Sci-Fi              226,277,068           8.1   
1     \nAction, Adventure, Fantasy              412,563,408           7.5   
2         \nAction, Drama, History              188,373,161           8.0   
3      \nAction, Adventure, Sci-Fi              389,813,101           7.7   
4     \nAction, Adventure, Fantasy              619,515,775           7.4   
5      \nAction, Adventure, Sci-Fi              334,201,140           7.5   
6      \nHorror, Mystery, Thriller              176,040,665           7.7   
7           \nAction, Crime, Music              107,825,862           7.7   
8      \nAction, Adventure, Comedy              315,040,301           8.0   
9         \nDrama, Mystery, Sci-Fi               92,054,159           8.1   
10       \nDrama, Horror, Thriller              327,481,748           7.5   
11       \nAction, Crime, Thriller               92,029,184           7.5   
12    \nAction, Adventure, Fantasy              229,010,985           6.8   
13      \nFamily, Fantasy, Musical              504,014,165           7.3   
14    \nAction, Adventure, Fantasy              168,052,812           6.7   
15      \nHorror, Sci-Fi, Thriller               74,262,031           6.5   
16    \nAction, Adventure, Fantasy              172,558,876           6.7   
17                  \nCrime, Drama               53,413,647           8.3   
18     \nAction, Adventure, Comedy              100,234,838           6.9   
19      \nAction, Adventure, Drama              146,880,162           7.5   
20     \nAdventure, Drama, Fantasy               61,182,734           7.6   
21      \nHorror, Sci-Fi, Thriller               30,234,022           6.6   
22          \nAction, Crime, Drama               40,533,014           6.4   
23      \nAction, Adventure, Drama               39,175,066           6.8   
24  \nAnimation, Adventure, Comedy              208,870,506           8.5   
25    \nAction, Adventure, Fantasy               80,101,125           5.5   
26      \nAction, Comedy, Thriller               75,468,583           6.9   
27        \nAction, Crime, Fantasy                     None           6.4   
28         \nC

#### Script for multiple pages
Building a script to scrape multiple pages can be a bit more challenging, we will have to build upon our old script by adding three more things:

Making all the requests we want from within the loop.
Controlling the loops rate to avoid bombarding the server with requests.
Monitoring the loop while it is in progress.
We'll scope through the first 4 pages of each year in the range 2010-2019

In [12]:

pages = [str(i) for i in range(1,6)] #creating list of strings corresponding to 4 pages
years = [str(i) for i in range(2010,2019)] #creating list corresponding to years 2000-2018

#### Controlling the crawl rate
If we avoid flooding the server with tens of request per second, then we are much likely to avoid our Ip being banned permenantly. We also avpid disrupting the activity of the website we scrape by allowing server to respond to other user's requests too.

We'll control the loop's rate by using the sleep() function in the python's "time" module. This will pause the execution of the loop for a specified amount of seconds.

To mimic the human behavious and to render our requests legit we will vary the amount of waiting time between requests by using the randint() function from python's "random" module

In [13]:
from time import sleep
from random import randint

#### Monitoring the loop as it's still going
Given that we have so many pages to scan through, it's better to have a way to moniter them while we are looping through them. This process in completely optional but is very helpful while debugging the process.If you are looping through say a 100+ pages, I'd say this is a must have feature.

For our script, we'll make sure to use this feature and measure the following parameters:

The frequency of requests, just to make sure we are not overloading the server.

The number of requests, so we can halt the loop incase the number of requests is exceeded.

The status code of our requests, so we make sure the server is sending back the correct responses.

In [14]:
from IPython.core.display import clear_output
from time import time

#redeclaring the variables

movie_names = []
year_release = []
imdb_ratings = []
metascores = []
votes = []
movie_description = []
certificate = []
runtime = []
genre = []
director_name = []
star_cast = []
gross_value = []

#preparing the moniter of the loop
start_time = time()
requests = 0

#for every year in the interval 2000-2018
for year in years:
    
    #for every page in the onterval 1-4
    for page in pages:
        
        #make a get request
        response = get("http://www.imdb.com/search/title?release_date=" + year + "&sort=num_votes,desc&page=" + page)
        
        #pause the loop
        sleep(randint(8,15))
        
        #monitor the requests
        requests += 1
        sleep(randint(1,3))
        elapsed_time = time() - start_time
        print("Request: {}, Frequency: {} requests/s".format(requests, requests/elapsed_time))
        print(page, year)
        clear_output(wait = True)
        
        #throw a warning for non-200 status codes
        if response.status_code != 200:
            warn("Request: {}, Status Code: {} ".format(requests, response.status_code))
            
            
        #break the loop if the frequency of request is too higih
        if requests > 101:
            warn("Number of requests was greater than expected.")
            break
            
        #parse the content through the html.parser using BeautifulSoup
        html_page = bs(response.text, "html.parser")
        
        #select all 50 movie container for a single page
        containers = html_page.find_all("div", class_ = "lister-item mode-advanced")
        
        #for every movie of the 50 movies
        for container in containers:
            
            #if movie has Metascore, then extract:
            if container.find("div", class_ = "ratings-metascore") is not None:
                
                #the movie_name
                name = container.h3.a.text
                movie_names.append(name)

                #the year od release
                release = container.find("span", class_ = "lister-item-year text-muted unbold").text
                year_release.append(release)

                #the ratings for the movies
                ratings = float(container.strong.text)
                imdb_ratings.append(ratings)

                #the metascores
                meta = container.find("span", class_ = "metascore").text
                metascores.append(int(meta))

                #the votes
                vote = container.find("span", attrs = {"name":"nv"})['data-value']
                votes.append(int(vote))

                #the certificate
                if container.find("span", class_ = "certificate") is not None:
                    certi = container.find("span", class_ = "certificate").text
                    certificate.append(certi)
                else:
                    certificate.append(None)

                #the runtime
                if container.find("span", class_ ="runtime") is not None:
                    run = container.find("span", class_ ="runtime").text
                    runtime.append(run)
                else:
                    runtime.append(None)

                #the genre
                if container.find("span", class_ ="genre") is not None:
                    gen = container.find("span", class_ ="genre").text
                    genre.append(gen)
                else:
                    genre.append(None)

                #fetching all <p> tags
                content = container.find_all("p")

                #the description
                if content[1] is not None:
                    desc = content[1].text
                    movie_description.append(desc)
                else:
                    movie_description.append(None)

                #subsetting all the <a> tags in 3rd <p> tag
                content_2 = content[2].find_all("a")

                #the director
                if content_2[0] is not None:
                    director = content_2[0].text
                    director_name.append(director)
                else:
                    director_name.append(None)

                #the gross value
                if len(container.find_all("span", attrs = {"name":"nv"})) >= 2:
                    gross = container.find_all("span", attrs = {"name":"nv"})[1]['data-value']
                    gross_value.append(gross)
                else:
                    gross_value.append(None)
                    
                #extracting artists names
                if content_2[1] is not None:
                    temp = []
                    for i in range(len(content_2)-1):
                        temp.append(content_2[i].text)
                    star_cast.append(temp)
                else:    
                    star_cast.append(None)

Request: 45, Frequency: 0.06508031892812906 requests/s
5 2018


In [15]:
#storing scraped data into a data frame.

imdb_movie_dataset = pd.DataFrame({"movie_names":movie_names,
                        "year_release":year_release,
                        "imdb_ratings":imdb_ratings,
                        "metscores":metascores,
                        "votes":votes,
                        "movie_description":movie_description,
                        "certificate":certificate,
                        "runtime":runtime,
                        "genre":genre,
                        "director_name": director_name,
                        "star_cast": star_cast,
                        "gross_value":gross_value
                       })

In [16]:
imdb_movie_dataset.to_csv("imdb_movie_dataset.csv") #stores the DataFrame as a csv file

In [17]:
imdb_movie_dataset = pd.read_csv("imdb_movie_dataset.csv") #reading the file we previously created.
import re

#the below code is used to clean the scraped data into a storable and further usable format. 
#We have used regular expressions in order to do so

for i in range(len(imdb_movie_dataset['star_cast'])):
    
    #cleaning star_cast column
    imdb_movie_dataset['star_cast'][i] = re.findall(r"'([^']*)'",imdb_movie_dataset['star_cast'][i])
    
    #cleaning genre column
    imdb_movie_dataset['genre'][i] = re.findall("([^\\r\\n\s,][a-zA-Z]+)", imdb_movie_dataset['genre'][i])
    
    #cleaning gross_value
    if (type(imdb_movie_dataset["gross_value"][i]) == float) is not True:
        imdb_movie_dataset['gross_value'][i] = int(imdb_movie_dataset['gross_value'][i].replace(",",""))
    else:
        imdb_movie_dataset['gross_value'][i] = None
    #cleaning movie_description
    imdb_movie_dataset['movie_description'][i] = str(re.findall(r"[^\\r\n].+",imdb_movie_dataset['movie_description'][i])[0])
    
    #cleaning movie_names
    imdb_movie_dataset["movie_names"][i] = str(imdb_movie_dataset["movie_names"][i][0])
    
    #cleaning runtime
    if (type(imdb_movie_dataset["runtime"][i])== float) is not True:
        imdb_movie_dataset["runtime"][i] = int(re.findall(r"[0-9].+[^a-zA-Z-]",imdb_movie_dataset['runtime'][i])[0])
    else:
        imdb_movie_dataset["runtime"][i] = None

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

#### Showing cleaned data and then saving it to csv file

In [18]:
imdb_movie_dataset

Unnamed: 0 certificate          director_name  \
0              0       PG-13      Christopher Nolan   
1              1           R        Martin Scorsese   
2              2           G            Lee Unkrich   
3              3           R       Darren Aronofsky   
4              4       PG-13            Jon Favreau   
5              5           R             Tom Hooper   
6              6          PG           Dean DeBlois   
7              7       PG-13          David Fincher   
8              8           R         Matthew Vaughn   
9              9          PG          Pierre Coffin   
10            10       PG-13            David Yates   
11            11          PG             Tim Burton   
12            12          PG           Nathan Greno   
13            13       PG-13           Edgar Wright   
14            14       PG-13             Will Gluck   
15            15           R            Danny Boyle   
16            16           R            Ben Affleck   
17            17           R       David O. Russell   
18            18           R     Sylvester Stallone   
19            19           R          Todd Phillips   
20            20          PG        Joseph Kosinski   
21            21       PG-13             Ethan Coen   
22            22       PG-13          Phillip Noyce   
23            23       PG-13       Robert Schwentke   
24            24           R          Albert Hughes   
25            25       PG-13        Louis Leterrier   
26            26       PG-13            Mike Newell   
27            27       PG-13              James Wan   
28            28       PG-13           Ridley Scott   
29            29       PG-13           Joe Carnahan   
...          ...         ...                    ...   
1495        1495       PG-13     Jaume Collet-Serra   
1496        1496       TV-MA           Duncan Jones   
1497        1497       PG-13           Adam Robitel   
1498        1498   Not Rated  Sanjay Leela Bhansali   
1499        1499           R     Christian Gudegast   
1500        1500           R             Lin Oeding   
1501        1501           R        Nicolai Fuglsig   
1502        1502       TV-MA             David Wain   
1503        1503          PG           Ava DuVernay   
1504        1504       PG-13         Clint Eastwood   
1505        1505       PG-13        Michael Spierig   
1506        1506           R               Eli Roth   
1507        1507          PG              Nick Park   
1508        1508       TV-MA       Martin Zandvliet   
1509        1509          PG             Will Gluck   
1510        1510         NaN        Stephanie Laing   
1511        1511           R          Brett Donowho   
1512        1512           R       Johannes Roberts   
1513        1513           R           Babak Najafi   
1514        1514           R           Peter Howitt   
1515        1515           R          Nash Edgerton   
1516        1516       PG-13          Greg Berlanti   
1517        1517       PG-13          Michael Sucsy   
1518        1518           R             Tim Hunter   
1519        1519          PG    Bethany Ashton Wolf   
1520        1520       PG-13              Rob Cohen   
1521        1521           R     Dimitri Logothetis   
1522        1522       PG-13        Bruce Macdonald   
1523        1523       PG-13           Wes Anderson   
1524        1524         NaN            James Marsh   

                               genre gross_value  imdb_ratings  metscores  \
0      [Action, Adventure, Sci, -Fi]   292576195           8.8         74   
1                [Mystery, Thriller]   128012934           8.1         63   
2     [Animation, Adventure, Comedy]   415004880           8.3         92   
3                  [Drama, Thriller]   106954678           8.0         79   
4      [Action, Adventure, Sci, -Fi]   312433331           7.0         57   
5                 [Biography, Drama]   138797449           8.0         88   
6     [Animation, Action, Adventure]   217581231        

In [19]:
imdb_movie_dataset.to_csv("Updated_imdb_cleaned_data.csv")

In [3]:
import pandas as pd
modifiedDataFile = pd.read_csv("Updated_imdb_cleaned_data.csv")

In [4]:
modifiedDataFile

Unnamed: 0  Unnamed: 0.1 certificate          director_name  \
0             0             0       PG-13      Christopher Nolan   
1             1             1       PG-13             James Gunn   
2             2             2           R          David Fincher   
3             3             3       PG-13          Anthony Russo   
4             4             4       PG-13           Bryan Singer   
5             5             5           R           Wes Anderson   
6             6             6       PG-13          Morten Tyldum   
7             7             7           R        Damien Chazelle   
8             8             8       PG-13             Doug Liman   
9             9             9           R         Matthew Vaughn   
10           10            10           R  Alejandro G. Iñárritu   
11           11            11       PG-13          Peter Jackson   
12           12            12       PG-13            Neil Burger   
13           13            13           R         Chad Stahelski   
14           14            14           R           Alex Garland   
15           15            15           R             Luc Besson   
16           16            16           R         Clint Eastwood   
17           17            17       PG-13            Matt Reeves   
18           18            18           R             Dan Gilroy   
19           19            19       PG-13              Marc Webb   
20           20            20       PG-13               Wes Ball   
21           21            21           R             David Ayer   
22           22            22       PG-13       Francis Lawrence   
23           23            23          PG               Don Hall   
24           24            24       PG-13         Gareth Edwards   
25           25            25       PG-13            James Marsh   
26           26            26           R      Richard Linklater   
27           27            27           R              Phil Lord   
28           28            28       PG-13             Josh Boone   
29           29            29          PG       Robert Stromberg   
..          ...           ...         ...                    ...   
716         716           716       PG-13               Wes Ball   
717         717           717           R            James Foley   
718         718           718       TV-14             Ari Sandel   
719         719           719       PG-13     Jaume Collet-Serra   
720         720           720       PG-13           Adam Robitel   
721         721           721           R           Alex Garland   
722         722           722           R     John Francis Daley   
723         723           723   Not Rated  Sanjay Leela Bhansali   
724         724           724         NaN           Duncan Jones   
725         725           725           R       Francis Lawrence   
726         726           726           R     Christian Gudegast   
727         727           727           R             Lin Oeding   
728         728           728           R        Nicolai Fuglsig   
729         729           729       TV-MA             David Wain   
730         730           730       PG-13         Clint Eastwood   
731         731           731       PG-13        Michael Spierig   
732         732           732          PG              Nick Park   
733         733           733          PG             Will Gluck   
734         734           734           R          Brett Donowho   
735         735           735           R               Eli Roth   
736         736           736         NaN        Stephanie Laing   
737         737           737           R           Babak Najafi   
738         738           738           R           Peter Howitt   
739         739           739          PG    Bethany Ashton Wolf   
740         740           740       PG-13          Michael Sucsy   
741         741           741           R             Tim Hunter   
742         742           742           R     Dimitri Logothe

In [176]:
Certificate = modifiedDataFile[['certificate']]
#Certificate = pd.DataFrame(Certificate,columns=['Certificate'])

In [177]:
type(Certificate)#.certificate.sum

pandas.core.frame.DataFrame

In [182]:
##Certificate = Certificate.certificate.unique
unique_cert = pd.unique(Certificate['certificate'].values)

In [183]:
unique_cert

array(['PG-13', 'R', 'PG', 'Not Rated', 'G', 'TV-14', 'TV-MA', nan], dtype=object)

In [184]:
for arr in unique_cert:
    print(arr)

PG-13
R
PG
Not Rated
G
TV-14
TV-MA
nan


In [189]:
type(unique_cert)

Certificate= unique_cert.tolist()

In [112]:
#Certificate = Certificate['certificate'].values.tolist()

In [190]:
type(Certificate)

list

In [191]:
we =1
results = []
id1 = []

In [196]:
#n = [[1, 2, 3], [4, 5, 6, 7, 8, 9]]
# Add your function here

for numbers in Certificate:
    results.append(numbers)
    id1.append(we)
    we=we+1

In [197]:
Certificate_DataFrame = pd.DataFrame({'Certificate_id':id1,'Certificate_Name':results})
#df = pd.DataFrame({'movies':moviesTest,'Hashtags':HashtagsTest})


In [200]:
type(Certificate_DataFrame) #= Certificate.sort

pandas.core.frame.DataFrame

In [201]:
Certificate_DataFrame

Certificate_Name  Certificate_id
0             PG-13               1
1                 R               2
2                PG               3
3         Not Rated               4
4                 G               5
5             TV-14               6
6             TV-MA               7
7               NaN               8
8             PG-13               9
9                 R              10
10               PG              11
11        Not Rated              12
12                G              13
13            TV-14              14
14            TV-MA              15
15              NaN              16

In [21]:
imdb_movie_dataset = pd.read_csv("imdb_movie_dataset.csv",encoding='latin-1') #reading the file we previously created.